In [1]:
import sys
import math
import numpy as np
import pandas as pd
sys.path.append('../src/')

In [2]:
s_fp = '/valhalla/fastq/2021.03.19/sd_county_samples.tsv'
s_df = pd.read_csv(s_fp, sep='\t')
s_df

,forward,reverse,sample,sample_library,sequencing_tech
0,/valhalla/fastq/2021.03.19/21-256229-SCV-02262...,/valhalla/fastq/2021.03.19/21-256229-SCV-02262...,21-256229-SCV-022621-AM,21-256229-SCV-022621-AM_L001,illumina
1,/valhalla/fastq/2021.03.19/21-256232-SCV-02262...,/valhalla/fastq/2021.03.19/21-256232-SCV-02262...,21-256232-SCV-022621-AM,21-256232-SCV-022621-AM_L001,illumina
2,/valhalla/fastq/2021.03.19/21-256234-SCV-02262...,/valhalla/fastq/2021.03.19/21-256234-SCV-02262...,21-256234-SCV-022621-AM,21-256234-SCV-022621-AM_L001,illumina
3,/valhalla/fastq/2021.03.19/21-256237-SCV-02262...,/valhalla/fastq/2021.03.19/21-256237-SCV-02262...,21-256237-SCV-022621-AM,21-256237-SCV-022621-AM_L001,illumina
4,/valhalla/fastq/2021.03.19/21-256246-SCV-02262...,/valhalla/fastq/2021.03.19/21-256246-SCV-02262...,21-256246-SCV-022621-AM,21-256246-SCV-022621-AM_L001,illumina
5,/valhalla/fastq/2021.03.19/21-256265-SCV-02262...,/valhalla/fastq/2021.03.19/21-256265-SCV-02262...,21-256265-SCV-022621-AM,21-256265-SCV-022621-AM_L001,illumina
6,/valhalla/fastq/2021.03.19/21-256267-SCV-02262...,/valhalla/fastq/2021.03.19/21-256267-SCV-02262...,21-256267-SCV-022621-AM,21-256267-SCV-022621-AM_L001,illumina
7,/valhalla/fastq/2021.03.19/21-256274-SCV-02262...,/valhalla/fastq/2021.03.19/21-256274-SCV-02262...,21-256274-SCV-022621-AM,21-256274-SCV-022621-AM_L001,illumina
8,/valhalla/fastq/2021.03.19/Fastq/21-170487-SCV...,/valhalla/fastq/2021.03.19/Fastq/21-170487-SCV...,21-170487-SCV2-031521-AM-1-V2,21-170487-SCV2-031521-AM-1-V2_L001,illumina
9,/valhalla/fastq/2021.03.19/Fastq/21-227289-SCV...,/valhalla/fastq/2021.03.19/Fastq/21-227289-SCV...,21-227289-SCV2-031521-AM-1-V2,21-227289-SCV2-031521-AM-1-V2_L001,illumina


In [6]:
s_df = s_df.loc[s_df['sample']!='21-299332-SCV2-031521-AM-1-V2']
s_df.to_csv(s_fp, sep='\t', index=False)

In [2]:
muts_fp = '/valhalla/gisaid/workflow/mutations_2021-03-19.csv'
muts = pd.read_csv(muts_fp)
# muts.loc[muts['gene'].str.contains('Non')]['gene']
# muts.loc[~(muts['gene']=='Non-coding region'), 'gene'].unique()

/home/al/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7,51) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
muts.loc[muts['date_collected']<=muts['date_submitted']]

,idx,seq_len,replacements,pos,gene,gene_start_pos,codon_num,codon_start,ref_codon,alt_codon,...,division_lower,location_original_lower,location_lower,tmp,date_modified,country_id,tmp_info1,division_id,tmp_info2,location_id
0,Ireland/G-NVRL-e84IRL94605/2021,29674,822:T,822,ORF1a,265,186,820.0,GTC,GTT,...,galway,none,none,"['2021', '02', '01']",2021-03-19-09:00,IRL,Ireland-Galway,GY,Ireland-Galway-None,None
1,Ireland/G-NVRL-e84IRL94605/2021,29674,912:T,912,ORF1a,265,216,910.0,TCC,TCT,...,galway,none,none,"['2021', '02', '01']",2021-03-19-09:00,IRL,Ireland-Galway,GY,Ireland-Galway-None,None
2,Ireland/G-NVRL-e84IRL94605/2021,29674,3036:T,3036,ORF1a,265,924,3034.0,TTC,TTT,...,galway,none,none,"['2021', '02', '01']",2021-03-19-09:00,IRL,Ireland-Galway,GY,Ireland-Galway-None,None
3,Ireland/G-NVRL-e84IRL94605/2021,29674,3266:T,3266,ORF1a,265,1001,3265.0,ACT,ATT,...,galway,none,none,"['2021', '02', '01']",2021-03-19-09:00,IRL,Ireland-Galway,GY,Ireland-Galway-None,None
4,Ireland/G-NVRL-e84IRL94605/2021,29674,5387:A,5387,ORF1a,265,1708,5386.0,GCT,GAT,...,galway,none,none,"['2021', '02', '01']",2021-03-19-09:00,IRL,Ireland-Galway,GY,Ireland-Galway-None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15150228,Belgium/ULG-10352/2020,29409,None,266,ORF1a,267,1,None,GGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACTC...,None,...,liège,none,none,"['2020', '08', '04']",2021-03-19-09:00,BEL,Belgium-Liège,None,Belgium-Liège-None,None
15150229,env/Uruguay/MO-1592/2020,29409,None,29666,ORF10,29559,37,None,CACATAGCA,None,...,montevideo,none,none,"['2020', '05', '08']",2021-03-19-09:00,URY,Uruguay-Montevideo,MO,Uruguay-Montevideo-None,None
15150230,USA/FL-BPHL-1244/2020,29409,None,29668,ORF10,29559,38,None,CATAGCA,None,...,florida,none,none,"['2020', '06', '02']",2021-03-19-09:00,USA,United States-Florida,FL,United States-Florida-None,None
15150231,India/MH-NIV-114444/2020,29409,None,25229,S,21564,1223,None,CTTGATTGCCATAGTA,None,...,maharashtra,none,none,"['2020', '07', '14']",2021-03-19-09:00,IND,India-Maharashtra,MH,India-Maharashtra-None,None


In [10]:
(muts.loc[muts['date_collected']>muts['date_submitted']][['strain', 'date_collected', 'date_submitted']]
 .drop_duplicates(subset=['strain'])).shape

(29, 3)

In [2]:
j_fp = '/home/al/analysis/gisaid/new_api_data.json.gz'
j_df = pd.read_json(j_fp, compression='gzip')

In [3]:
print(j_df.shape)
print(j_df['accession_id'].unique().shape)

(754926, 24)
(754926,)


In [2]:
%load_ext autoreload
%autoreload 2

import bjorn_support as bs
import mutations as bm
# import unsupervised_learning as bul

In [3]:
m_fp = '/valhalla/gisaid/workflow/metadata_2021-03-14.tsv.gz'
df = bs.create_chunk_names(m_fp, 10000)

In [4]:
df

,chunk_names
0,chunk_1
1,chunk_2
2,chunk_3
3,chunk_4
4,chunk_5
...,...
73,chunk_74
74,chunk_75
75,chunk_76
76,chunk_77


In [ ]:
def create_chunk_names(meta_filepath: str, chunk_size: int) -> pd.DataFrame:
    # with open(fasta_filepath, 'r') as filehandle:
    #     count = 0
    #     for line in filehandle:
    #         if line.startswith(">"):
    #             count += 1
    meta_df = pd.read_csv(meta_filepath, sep='\t', compression='gzip')
    num_sequences = meta_df['strain'].unique().shape[0]
    chunk_names = [f"chunk_{i+1}" for i in range(math.ceil(num_sequences / chunk_size))]
    return pd.DataFrame(data=chunk_names, columns=['chunk_names'])

In [18]:
from path import Path
t = "chunks/fasta/date/sample.fasta"
Path(t).parent

Path('chunks/fasta/date')

In [11]:
# meta_df = pd.read_csv(m_fp, sep='\t', compression='gzip')
# num_sequences = meta_df['accession_id'].unique().shape[0]

In [10]:
num_sequences

770064

In [16]:
meta_df['strain'].unique().shape[0]

769793

In [14]:
760000 + 9794

769794

In [8]:
chunk_names = [f"chunk_{i+1}" for i in range(math.ceil(num_sequences / 10000))]

In [9]:
chunk_names

['chunk_1',
 'chunk_2',
 'chunk_3',
 'chunk_4',
 'chunk_5',
 'chunk_6',
 'chunk_7',
 'chunk_8',
 'chunk_9',
 'chunk_10',
 'chunk_11',
 'chunk_12',
 'chunk_13',
 'chunk_14',
 'chunk_15',
 'chunk_16',
 'chunk_17',
 'chunk_18',
 'chunk_19',
 'chunk_20',
 'chunk_21',
 'chunk_22',
 'chunk_23',
 'chunk_24',
 'chunk_25',
 'chunk_26',
 'chunk_27',
 'chunk_28',
 'chunk_29',
 'chunk_30',
 'chunk_31',
 'chunk_32',
 'chunk_33',
 'chunk_34',
 'chunk_35',
 'chunk_36',
 'chunk_37',
 'chunk_38',
 'chunk_39',
 'chunk_40',
 'chunk_41',
 'chunk_42',
 'chunk_43',
 'chunk_44',
 'chunk_45',
 'chunk_46',
 'chunk_47',
 'chunk_48',
 'chunk_49',
 'chunk_50',
 'chunk_51',
 'chunk_52',
 'chunk_53',
 'chunk_54',
 'chunk_55',
 'chunk_56',
 'chunk_57',
 'chunk_58',
 'chunk_59',
 'chunk_60',
 'chunk_61',
 'chunk_62',
 'chunk_63',
 'chunk_64',
 'chunk_65',
 'chunk_66',
 'chunk_67',
 'chunk_68',
 'chunk_69',
 'chunk_70',
 'chunk_71',
 'chunk_72',
 'chunk_73',
 'chunk_74',
 'chunk_75',
 'chunk_76',
 'chunk_77',
 'chunk_

In [3]:
m_fp = '/valhalla/gisaid/workflow/metadata_2021-03-13.tsv.gz'
meta = pd.read_csv(m_fp, sep='\t', compression='gzip')

In [4]:
meta

,strain,covsurver_prot_mutations,location,pangolin_lineage,date_collected,accession_id,pangolin_version,clade,date_submitted,country,division,country_normed,division_normed,location_normed,country_lower,country_normed_lower,division_lower,division_normed_lower,location_lower,location_normed_lower
0,Australia/NT12/2020,"(NSP15_A283V,NSP12_P323L,Spike_D614G)",None,B.1,2020,EPI_ISL_426900,2021-02-21,G,2020-04-17,Australia,Northern territory,Australia,Northern Territory,None,australia,australia,northern territory,northern territory,none,none
1,Australia/NT13/2020,"(NSP15_A283V,NSP12_P323L,Spike_D614G)",None,B.1,2020,EPI_ISL_426901,2021-02-21,G,2020-04-17,Australia,Northern territory,Australia,Northern Territory,None,australia,australia,northern territory,northern territory,none,none
2,Australia/NT14/2020,"(NSP14_R163C,NSP3_K38R,NS3_G251V,NSP2_I559V,NS...",None,B.40,2020,EPI_ISL_426902,2021-02-21,V,2020-04-17,Australia,Northern territory,Australia,Northern Territory,None,australia,australia,northern territory,northern territory,none,none
3,Australia/NT16/2020,"(NSP12_P323L,Spike_D614G)",None,B.1.8,2020,EPI_ISL_426903,2021-02-21,G,2020-04-17,Australia,Northern territory,Australia,Northern Territory,None,australia,australia,northern territory,northern territory,none,none
4,Australia/NT17/2020,"(NSP12_P323L,Spike_D614G)",None,B.1.8,2020,EPI_ISL_426904,2021-02-21,G,2020-04-17,Australia,Northern territory,Australia,Northern Territory,None,australia,australia,northern territory,northern territory,none,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764756,USA/ND-NDDH-0345/2021,"(N_D377Y,NSP3_L72F,N_P67S,NSP16_R216C,NS3_Q57H...",None,B.1.2,2021-01-07,EPI_ISL_1233725,2021-02-21,GH,2021-03-12,USA,North Dakota,United States,North Dakota,None,usa,united states,north dakota,north dakota,none,none
764757,USA/ND-NDDH-0346/2021,"(NSP15_A117T,NSP3_K837R,N_S194L,NSP5_P96L,NSP7...",None,B.1.400,2021-01-07,EPI_ISL_1233726,2021-02-21,G,2021-03-12,USA,North Dakota,United States,North Dakota,None,usa,united states,north dakota,north dakota,none,none
764758,USA/ND-NDDH-0347/2021,"(N_D377Y,NSP3_K837R,NSP3_L1301F,Spike_S494P,NS...",None,B.1.2,2021-01-07,EPI_ISL_1233727,2021-02-21,GH,2021-03-12,USA,North Dakota,United States,North Dakota,None,usa,united states,north dakota,north dakota,none,none
764759,USA/ND-NDDH-0348/2021,"(NSP3_P968L,Spike_H69del,Spike_A67V,N_A12G,E_L...",None,B.1.525,2021-02-26,EPI_ISL_1233728,2021-02-21,G,2021-03-12,USA,North Dakota,United States,North Dakota,None,usa,united states,north dakota,north dakota,none,none


In [25]:
api_data_fp = 'new_api_data.json.gz'
gcloud_stat_cmd = f'/home/al/code/google-cloud-sdk/bin/gsutil stat gs://andersen-lab_temp/outbreak_genomics/new_api_data.json.gz'
t = bs.run_command_log(gcloud_stat_cmd)
t

'gs://andersen-lab_temp/outbreak_genomics/new_api_data.json.gz:\n    Creation time:          Fri, 12 Mar 2021 18:29:33 GMT\n    Update time:            Fri, 12 Mar 2021 18:29:38 GMT\n    Storage class:          MULTI_REGIONAL\n    Content-Language:       en\n    Content-Length:         130872684\n    Content-Type:           application/json\n    Hash (crc32c):          YwJP2A==\n    Hash (md5):             99G3k2Qh2pSuj/KCZtm2Iw==\n    ETag:                   CIa3i6exq+8CEAI=\n    Generation:             1615573773638534\n    Metageneration:         2\n'

In [13]:
t

In [4]:
fp = '/valhalla/gisaid/workflow/mutations_2021-03-09.csv'
muts = pd.read_csv(fp)

/home/al/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7,35,41,43,51) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
muts.loc[muts['country']=='United States', 'location_id']

25605        6085
25606        6085
25607        6085
25608        6085
25609        6085
            ...  
12838055    48225
12838056    48225
12838057    48225
12838058    48225
12838059    48225
Name: location_id, Length: 2516606, dtype: object

In [68]:
muts = muts[muts['is_synonymous']==False]

In [125]:
lineage = 'B.1.526'
data = (muts.loc[muts['pangolin_lineage']==lineage, 'mutation']
        .value_counts()
        .to_frame()
        .reset_index()
        .rename(columns={'index': 'name', 'mutation': 'count'}))#.values
data['frequency'] = (data['count'] - np.min(data['count']))/np.ptp(data['count'])
data = data[data['frequency']>=0.05]
# data

In [126]:
from scipy.spatial import distance
from scipy.stats import mode
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import SpectralClustering
from sklearn.metrics import confusion_matrix, accuracy_score
from skimage.io import imread

In [127]:
def kmeans_unsupervised(X:np.ndarray, k:int, centroids=None, tolerance=1e-2):
    """K-Means Clustering with unsupervised learning trick to infer characteristic mutations
    for a given SARS-CoV-2 lineage"""
    # Initialize Input, Centroids, Clusters
    if len(X.shape) < 2: X = np.expand_dims(X, axis=1)
    if centroids: 
        centroids = bul.init_centroids(X, k)
    else:
        centroids = X[np.random.choice(X.shape[0], size=k, replace=False), :]
    clusters = [[] for c in centroids]
    # Until Convergence
    while(True):
        # Centroid Norm at t-1
        old_centroids_norm = np.linalg.norm(centroids)
        # Minimum Distance to Centroid Per Record
        js = np.argmin(distance.cdist(X, centroids), axis=1)
        # Assign Records to Clusters
        for i, c in enumerate(clusters):
             clusters[i] = list(np.where(js == i)[0])
        # Recompute Cluster Centroids
        for j, c in enumerate(centroids):
            centroids[j] = np.mean(X[clusters[j]], axis=0)
        # Centroid Norm at t
        centroids_norm = np.linalg.norm(centroids)
        # Convergence!
        if np.abs(centroids_norm - old_centroids_norm) < tolerance:
            preds = np.zeros_like(X)
            clusters = np.asarray(clusters)[np.argsort(centroids.squeeze())[::-1]]
#             return clusters, centroids
            for i, c in enumerate(clusters):
                preds[c] = i
            return preds.squeeze()

In [128]:
X = np.expand_dims(data['frequency'].to_numpy(), axis=1)
preds = kmeans_unsupervised(X, k=2, centroids="kmeans++") == 0
preds

/Users/al/anaconda3/envs/bjorn/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False, False, False,
       False, False, False, False, False, False])

In [129]:
# data['is_characteristic'] = bul.spectral_clustering_unsupervised(X, num_classes=2, n_trees=100)

In [130]:
data['is_characteristic'] = preds
data['lineage'] = lineage
data.to_csv(f'/Users/al/Documents/scripps/analysis/bjorn/{lineage}_char_mutations.csv', index=False)

In [131]:
data

,name,count,frequency,is_characteristic,lineage
0,S:D614G,1621,1.000000,True,B.1.526
1,S:D253G,1620,0.999383,True,B.1.526
2,ORF1b:Q1011H,1618,0.998148,True,B.1.526
3,ORF1b:P314L,1617,0.997531,True,B.1.526
4,ORF1a:T265I,1611,0.993827,True,B.1.526
5,ORF1a:L3201P,1606,0.990741,True,B.1.526
6,S:T95I,1602,0.988272,True,B.1.526
7,ORF3a:Q57H,1591,0.981481,True,B.1.526
8,ORF1a:DEL3675/3677,1574,0.970988,True,B.1.526
9,S:L5F,1561,0.962963,True,B.1.526
